<a href="https://colab.research.google.com/github/lucashamon1/Detec-o-de-Fraudes-com-Cart-o-de-Cr-dito/blob/main/Fraude_com_Cart%C3%A3o_de_Cr%C3%A9dito.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

A ideia do projeto é usar árvore de decisão a fim de detectar fraudes nos gastos do cartão de crédito e movimentações estranhas

In [1]:
import pandas as pd
dados = pd.read_csv('creditcard.csv')
dados.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0.0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0.0
2,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0.0
3,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0.0
4,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0.0


#Analise Prévia

In [2]:
dados = dados.dropna()

In [3]:
n_transaçoes = dados['Class'].count()
print(n_transaçoes)

37721


In [4]:
n_fraudes = dados['Class'].sum()
print(n_fraudes)

103.0


In [5]:
n_normais = n_transaçoes - n_fraudes
porcentagem = (n_fraudes / n_transaçoes)
porcentagem_normais = (n_normais / n_transaçoes) * 100
porcentagem_fraudes = (n_fraudes / n_transaçoes) * 100
print('transações normais, em %: ',(porcentagem_normais))
print('transações com fraqude, em %: ',(porcentagem_fraudes)) 

transações normais, em %:  99.72694255189417
transações com fraqude, em %:  0.27305744810582966


Temos um total de 21877 transações, sendo que 86 delas são fraudes. Ou seja, 99,6% das transações são normais e 0,4% delas são fraudulentas 

#Criando dados de treino e dados de teste

In [6]:
from sklearn.model_selection import StratifiedShuffleSplit
x =dados.drop('Class', axis=1).values
y = dados['Class'].values

In [13]:
def validador(x, y):
  validador = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=0)
  for treino_id, teste_id in validador.split(x, y):
    x_treino, x_teste = x[treino_id], x[teste_id]
    y_treino, y_teste = y[treino_id], y[teste_id]
    return x_treino, x_teste, y_treino, y_teste

In [8]:
%%time
from sklearn import tree
def classificador(validador, x_treino, x_teste, y_treino):
  classificador = tree.DecisionTreeClassifier()
  arvore = classificador.fit(x_treino, y_treino)
  y_pred = arvore.predict(x_teste)
  return y_pred

CPU times: user 29.8 ms, sys: 10.2 ms, total: 40 ms
Wall time: 202 ms


#Árvore de Decisão

In [22]:
import matplotlib.pyplot as plt
def salvar_arvore (classificador, nome):
  plt.figure(figsize=(200,100))
  tree.plot_tree(classificador, filled=True, fontsize=14)
  plt.savefig(nome)

In [24]:
X = dados.drop('Class', axis=1).values
Y = dados['Class'].values
x_treino, x_teste, y_treino, y_teste = validador(X, Y)
classificador_arvore_decisao = tree.DecisionTreeClassifier()
y_pred_decisao = classificador(classificador_arvore_decisao, x_treino, x_teste, y_treino)

In [25]:
classificador_arvore_decisao.fit(x_treino, y_treino)

DecisionTreeClassifier()

In [26]:
salvar_arvore(classificador_arvore_decisao, "arvore_decisao1.png")

#Validação do Modelo

Acurácia

In [67]:
%time
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

def validar_arvore(y_test, y_pred):
  print('acurácia: {}'.format(accuracy_score(y_test, y_pred)))
  print('precisão: {}'.format(precision_score(y_test, y_pred)))
  print('recall: {}'.format(recall_score(y_test, y_pred)))
  print('matriz de confusão {}'.format(confusion_matrix(y_test, y_pred)))

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.48 µs


In [38]:
validar_arvore(y_teste, y_pred_decisao)

acurácia: 0.9986747945931619
precisão: 0.6923076923076923
recall: 0.9
matriz de confusão [[3759    4]
 [   1    9]]


acurácia = 99.8%

Precisão = 69.2%

Recall = 90%

Ajustando os Parâmetros

In [42]:
classificador_arvore_decisao = tree.DecisionTreeClassifier(max_depth=10, random_state=0)

In [43]:
y_pred_decisao = classificador(classificador_arvore_decisao, x_treino, x_teste, y_treino)

In [44]:
validar_arvore(y_teste, y_pred_decisao)

acurácia: 0.9986747945931619
precisão: 0.6923076923076923
recall: 0.9
matriz de confusão [[3759    4]
 [   1    9]]


In [45]:
classificador_arvore_decisao = tree.DecisionTreeClassifier(max_depth=10, random_state=0, min_samples_leaf=10)
y_pred_decisao = classificador(classificador_arvore_decisao, x_treino, x_teste, y_treino)

In [46]:
validar_arvore(y_teste, y_pred_decisao)

acurácia: 0.9986747945931619
precisão: 0.6923076923076923
recall: 0.9
matriz de confusão [[3759    4]
 [   1    9]]


A modificação dos parâmetros não altera os resultados de árvore de decisão

#Modelos Ensemble

In [66]:
%time
from sklearn.ensemble import RandomForestClassifier
classificador_random_forest = RandomForestClassifier(n_estimators=100, random_state=0)
y_pred_random_forest = classificador(classificador_random_forest, x_treino, x_teste, y_treino)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.11 µs


In [57]:
validar_arvore(y_teste, y_pred_random_forest)

acurácia: 0.9986747945931619
precisão: 0.6923076923076923
recall: 0.9
matriz de confusão [[3759    4]
 [   1    9]]


Os resultados continuam os mesmos

In [60]:
classificador_random_forest = RandomForestClassifier(n_estimators=1000, random_state=0)
y_pred_random_forest = classificador(classificador_random_forest, x_treino, x_teste, y_treino)

In [61]:
validar_arvore(y_teste, y_pred_random_forest)

acurácia: 0.9986747945931619
precisão: 0.6923076923076923
recall: 0.9
matriz de confusão [[3759    4]
 [   1    9]]


#AdaBoost

In [65]:
%time
from sklearn.ensemble import AdaBoostClassifier
classificador_ada_boost = AdaBoostClassifier(random_state=0)
y_pred_adaboost = classificador(classificador_ada_boost, x_treino, x_teste, y_treino)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 13.6 µs


In [64]:
validar_arvore(y_teste, y_pred_adaboost)

acurácia: 0.9986747945931619
precisão: 0.6923076923076923
recall: 0.9
matriz de confusão [[3759    4]
 [   1    9]]


In [70]:
%time
from sklearn.ensemble import AdaBoostClassifier
classificador_ada_boost = AdaBoostClassifier(random_state=0, n_estimators=200)
y_pred_adaboost = classificador(classificador_ada_boost, x_treino, x_teste, y_treino)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.72 µs


In [71]:
validar_arvore(y_teste, y_pred_adaboost)

acurácia: 0.9986747945931619
precisão: 0.6923076923076923
recall: 0.9
matriz de confusão [[3759    4]
 [   1    9]]
